<a href="https://colab.research.google.com/github/lukasz86s/Machine_learning/blob/main/housing_regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get data

In [ ]:
import pandas as pd
raw_data = pd.read_csv('housing.csv')

In [8]:
dataset = raw_data.copy()

#Prepare data

In [9]:
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [10]:
dataset.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [11]:
dataset.dropna(inplace=True)

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [13]:
dataset.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [14]:
dataset.describe(include=['object'])

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [15]:
import plotly.express as px 
px.histogram(data_frame=dataset, x='median_house_value')

In [16]:
dataset.median_house_value.value_counts()

500001.0    958
137500.0    119
162500.0    116
112500.0    103
187500.0     92
           ... 
420500.0      1
419000.0      1
443000.0      1
343600.0      1
319500.0      1
Name: median_house_value, Length: 3833, dtype: int64

In [17]:
idx_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index = idx_to_drop)


In [18]:
px.histogram(x=dataset.median_house_value)

In [19]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True)
dataset_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,1,0


#Split data to train and test samples

In [20]:
train_data = dataset_dummies.sample(frac=0.8, random_state=0)
test_data = dataset_dummies.drop(train_data.index)

print(f'train data lenght: {train_data.shape[0]}')
print(f'test data lenght: {test_data.shape[0]}')

train data lenght: 15580
test data lenght: 3895


In [21]:
train_stats = train_data.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.0,-119.558780,2.006237,-124.3500,-121.750000,-118.4900,-117.990000,-114.4700
latitude,15580.0,35.648614,2.147016,32.5500,33.930000,34.2700,37.730000,41.9500
housing_median_age,15580.0,28.489217,12.505895,1.0000,18.000000,29.0000,37.000000,52.0000
total_rooms,15580.0,2620.014506,2195.654212,2.0000,1441.000000,2112.0000,3119.000000,39320.0000
total_bedrooms,15580.0,539.900578,424.499548,2.0000,299.000000,436.0000,647.000000,6445.0000
population,15580.0,1441.193068,1160.542775,3.0000,801.000000,1179.0000,1746.000000,35682.0000
households,15580.0,501.099487,385.039835,2.0000,283.000000,411.0000,605.000000,6082.0000
median_income,15580.0,3.672427,1.570297,0.4999,2.519975,3.4405,4.581425,15.0001
ocean_proximity_INLAND,15580.0,0.333569,0.471503,0.0000,0.000000,0.0000,1.000000,1.0000
ocean_proximity_ISLAND,15580.0,0.000193,0.013876,0.0000,0.000000,0.0000,0.000000,1.0000


In [22]:
train_labels = train_data.pop('median_house_value')
test_labels = test_data.pop('median_house_value')

In [23]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

In [24]:
normed_train_data = norm(train_data)
normed_test_data = norm(test_data)

In [25]:
normed_train_data.isnull().sum()

longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms                0
population                    0
households                    0
median_income                 0
ocean_proximity_INLAND        0
ocean_proximity_ISLAND        0
ocean_proximity_NEAR BAY      0
ocean_proximity_NEAR OCEAN    0
dtype: int64

In [26]:
normed_train_data = normed_train_data.values
normed_test_data = normed_test_data.values


#Create model

In [27]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=1024,kernel_regularizer='l2',activation='relu', input_shape=(normed_train_data.shape[1],)))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              13312     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 603,905
Trainable params: 603,905
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse', 'mae'])

#Train model

In [31]:
history = model.fit(x=normed_train_data, y=train_labels.values,epochs=150, validation_split=0.2, verbose=1, batch_size=32 )

Epoch 1/150
390/390 [==============================] - 6s 14ms/step - loss: 17050942464.0000 - mse: 17050942464.0000 - mae: 95150.4297 - val_loss: 4595264000.0000 - val_mse: 4595264000.0000 - val_mae: 49237.1367
Epoch 2/150
390/390 [==============================] - 5s 13ms/step - loss: 4195579392.0000 - mse: 4195579392.0000 - mae: 46448.0000 - val_loss: 3785122816.0000 - val_mse: 3785122816.0000 - val_mae: 44449.2930
Epoch 3/150
390/390 [==============================] - 5s 13ms/step - loss: 3756373760.0000 - mse: 3756373760.0000 - mae: 44062.0547 - val_loss: 3590438912.0000 - val_mse: 3590438912.0000 - val_mae: 43796.0977
Epoch 4/150
390/390 [==============================] - 5s 14ms/step - loss: 3623379200.0000 - mse: 3623379200.0000 - mae: 43244.7305 - val_loss: 3498175488.0000 - val_mse: 3498175488.0000 - val_mae: 42419.3359
Epoch 5/150
390/390 [==============================] - 6s 15ms/step - loss: 3544493056.0000 - mse: 3544493056.0000 - mae: 42486.3594 - val_loss: 3451244288.00

In [47]:
model.save('model_e150.h5')

([TensorSpec(shape=(None, 12), dtype=tf.float32, name='dense_input')], {})

In [61]:
history = history.history

In [62]:
import plotly.graph_objects as go
figure = go.Figure()
figure.add_trace(go.Scatter(y=history['mse'], name='mse'))
figure.add_trace(go.Scatter(y=history['val_mse'], name='val_mse'))
figure.update_xaxes(title='epochs')
figure.update_layout(width=1600, title='MSE')
figure.show()


In [60]:
from tensorflow.keras.models import load_model
model = load_model('model_e150.h5')

In [63]:
pred = model.predict(normed_test_data)

#Show scatterplot

In [76]:
fig = px.scatter(x=test_labels.values, y=pred)
fig.add_trace(go.Scatter(x=[0, 500_000], y=[0, 500_000], mode='lines'))
fig.show()

#Show error

In [156]:
df = pd.DataFrame({'mean_house_value': test_labels.values, 'predictions': pred.ravel() })
df['error'] = df.mean_house_value - df.predictions

In [157]:
df.head()

,mean_house_value,predictions,error
0,281500.0,226303.453125,55196.546875
1,191300.0,190167.906250,1132.093750
2,162900.0,174312.453125,-11412.453125
3,93800.0,149072.343750,-55272.343750
4,155400.0,178696.687500,-23296.687500


In [159]:
px.histogram(data_frame=df, x='error', width=1000 ,marginal='rug')